In [ ]:
# ============================================
# 📦 Step 1: Import Libraries
# ============================================

import warnings
warnings.filterwarnings('ignore')

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
import cv2
import random
from scipy.ndimage import rotate as scipy_rotate
from tqdm import tqdm
from pathlib import Path
import kagglehub

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✅ Libraries imported successfully")

In [ ]:
# ============================================
# 📁 Step 2: Download and Load Dataset
# ============================================

path = kagglehub.dataset_download("khanfashee/nih-chest-x-ray-14-224x224-resized")
BASE_PATH = Path(path)

df_labels = pd.read_csv(BASE_PATH / "Data_Entry_2017.csv")
images_dir = BASE_PATH / "images-224" / "images-224"
df_labels["Image Path"] = [str(images_dir / p) for p in df_labels["Image Index"].values]

DISEASE_CATEGORIES = [
    'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass',
    'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema',
    'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia'
]

for disease in DISEASE_CATEGORIES:
    df_labels[disease] = df_labels['Finding Labels'].apply(lambda x: 1 if disease in x else 0)

sample_paths = df_labels['Image Path'].sample(200, random_state=42).values
missing = [p for p in sample_paths if not os.path.exists(p)]
if missing:
    raise FileNotFoundError(f"❌ Missing {len(missing)} images!")

print(f"✅ Loaded {len(df_labels):,} images")

In [ ]:
# ============================================
# ⚙️ Step 3: Configuration
# ============================================

class Config:
    img_size = 224
    feat_dim = 256
    proj_dim = 128
    
    batch_size = 64
    pretrain_epochs = 50
    finetune_epochs = 30
    lr_pretrain = 1e-3
    lr_finetune = 1e-4
    temperature = 0.1
    
    num_workers = 4
    use_subset = False
    
    # Attention parameters
    num_regions = 6
    attention_layers = [2, 3, 4]
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cfg = Config()

print("⚙️ Configuration:")
print(f"   Device: {cfg.device}")
print(f"   Num regions: {cfg.num_regions}")
print(f"   Attention layers: {cfg.attention_layers}")

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True

In [ ]:
# ============================================
# 💾 Step 3.5: Checkpoint & Resume Configuration
# ============================================

import os
import shutil
from datetime import datetime

OPTION_NAME = "option5"

# ===== RESUME CONFIGURATION =====
CHECKPOINT_DATASET_NAME = "chest-xray-ssl-checkpoints"
RESUME_SSL_PRETRAINING = False
RESUME_FINETUNING = False
SSL_CHECKPOINT_FILE = "latest"
FINETUNE_CHECKPOINT_FILE = "latest"

IN_KAGGLE = os.path.exists('/kaggle')
IN_COLAB = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    CHECKPOINT_DIR = '/content/drive/MyDrive/chest_xray_ssl'
    IN_COLAB = True
except:
    pass

if IN_KAGGLE:
    CHECKPOINT_DIR = '/kaggle/working/checkpoints'
    PREV_CHECKPOINT_DIR = f'/kaggle/input/{CHECKPOINT_DATASET_NAME}'
    if os.path.exists(PREV_CHECKPOINT_DIR):
        print(f"✅ Found checkpoints at: {PREV_CHECKPOINT_DIR}")
        os.makedirs(CHECKPOINT_DIR, exist_ok=True)
        for f in os.listdir(PREV_CHECKPOINT_DIR):
            if f.endswith('.pth'):
                src, dst = os.path.join(PREV_CHECKPOINT_DIR, f), os.path.join(CHECKPOINT_DIR, f)
                if not os.path.exists(dst): shutil.copy2(src, dst)
elif not IN_COLAB:
    CHECKPOINT_DIR = './checkpoints'

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def save_checkpoint(state, filename):
    filepath = os.path.join(CHECKPOINT_DIR, filename)
    state['saved_at'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    torch.save(state, filepath)
    print(f"💾 Saved: {filename}")
    if IN_KAGGLE: torch.save(state, f'/kaggle/working/{filename}')

def load_checkpoint(filename):
    filepath = os.path.join(CHECKPOINT_DIR, filename)
    if os.path.exists(filepath):
        checkpoint = torch.load(filepath, map_location=cfg.device)
        print(f"✅ Loaded: {filename}")
        return checkpoint
    return None

def find_latest_checkpoint(prefix):
    if not os.path.exists(CHECKPOINT_DIR): return None
    latest = f'{prefix}_latest.pth'
    if os.path.exists(os.path.join(CHECKPOINT_DIR, latest)): return latest
    import re
    pattern = re.compile(rf'{prefix}_epoch(\d+)\.pth')
    max_epoch, best = -1, None
    for f in os.listdir(CHECKPOINT_DIR):
        m = pattern.match(f)
        if m and int(m.group(1)) > max_epoch: max_epoch, best = int(m.group(1)), f
    return best

print(f"🔧 Environment: {'Kaggle' if IN_KAGGLE else 'Colab' if IN_COLAB else 'Local'}")
print(f"📂 Checkpoint dir: {CHECKPOINT_DIR}")

In [ ]:
# ============================================
# 🫁 Step 4: Multi-Region Segmentation
# ============================================

def multi_region_segmentation(image):
    """
    Divide image into 6 anatomical regions:
    - Upper-Left, Upper-Right
    - Middle-Left, Middle-Right
    - Lower-Left, Lower-Right
    """
    if len(image.shape) == 3 and image.shape[0] == 1:
        image = image[0]
    
    h, w = image.shape
    regions = {}
    
    h_third = h // 3
    w_half = w // 2
    
    region_names = [
        'upper_left', 'upper_right',
        'middle_left', 'middle_right',
        'lower_left', 'lower_right'
    ]
    
    for i, name in enumerate(region_names):
        regions[name] = np.zeros_like(image)
    
    regions['upper_left'][0:h_third, 0:w_half] = 1
    regions['upper_right'][0:h_third, w_half:w] = 1
    regions['middle_left'][h_third:2*h_third, 0:w_half] = 1
    regions['middle_right'][h_third:2*h_third, w_half:w] = 1
    regions['lower_left'][2*h_third:h, 0:w_half] = 1
    regions['lower_right'][2*h_third:h, w_half:w] = 1
    
    return regions


def visualize_regions(image):
    """Visualize regions with different colors"""
    if len(image.shape) == 3 and image.shape[0] == 1:
        image = image[0]
    
    regions = multi_region_segmentation(image)
    h, w = image.shape
    colored = np.zeros((h, w, 3))
    
    colors = {
        'upper_left': [1, 0, 0],
        'upper_right': [0, 1, 0],
        'middle_left': [0, 0, 1],
        'middle_right': [1, 1, 0],
        'lower_left': [1, 0, 1],
        'lower_right': [0, 1, 1]
    }
    
    for region_name, color in colors.items():
        mask = regions[region_name] > 0.5
        colored[mask] = np.array(color)
    
    return colored

print("✅ Region segmentation functions defined")

In [ ]:
# ============================================
# 👁️ Step 5: Visualize Region Segmentation
# ============================================

sample_indices = [0, 100, 500]
fig, axes = plt.subplots(len(sample_indices), 3, figsize=(14, 4*len(sample_indices)))

for i, idx in enumerate(sample_indices):
    img_path = df_labels.iloc[idx]['Image Path']
    img = Image.open(img_path).convert('L')
    img = img.resize((cfg.img_size, cfg.img_size), Image.LANCZOS)
    img_np = np.array(img, dtype=np.float32) / 255.0
    
    axes[i, 0].imshow(img_np, cmap='gray')
    axes[i, 0].set_title('Original Image')
    axes[i, 0].axis('off')
    
    # Region masks
    regions = multi_region_segmentation(img_np)
    region_vis = visualize_regions(img_np)
    axes[i, 1].imshow(region_vis)
    axes[i, 1].set_title('6 Anatomical Regions (Color-Coded)')
    axes[i, 1].axis('off')
    
    # Overlay
    overlay = np.stack([img_np, img_np, img_np], axis=2)
    overlay = 0.7 * overlay + 0.3 * region_vis
    axes[i, 2].imshow(overlay)
    axes[i, 2].set_title('Overlay')
    axes[i, 2].axis('off')

plt.suptitle('Option 5: Multi-Region Attention Guidance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('option5_region_attention.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Region visualization complete")

In [ ]:
# ============================================
# 🔄 Step 6: Context-Aware Augmentation
# ============================================

def apply_augmentation(image):
    """Standard augmentation"""
    # Rotation
    if random.random() > 0.5:
        angle = random.uniform(-10, 10)
        image = scipy_rotate(image, angle, reshape=False, mode='nearest')
    
    # Horizontal flip
    if random.random() > 0.5:
        image = np.fliplr(image)
    
    # Brightness
    if random.random() > 0.5:
        factor = random.uniform(0.8, 1.2)
        image = np.clip(image * factor, 0, 1)
    
    # Contrast
    if random.random() > 0.5:
        mean = image.mean()
        factor = random.uniform(0.8, 1.2)
        image = np.clip((image - mean) * factor + mean, 0, 1)
    
    # Gaussian noise
    if random.random() > 0.6:
        noise = np.random.normal(0, 0.02, image.shape)
        image = np.clip(image + noise, 0, 1)
    
    return image

print("✅ Augmentation function defined")

In [ ]:
# ============================================
# 📦 Step 7: Dataset Classes
# ============================================

class AttentionGuidedDataset(Dataset):
    """SSL dataset with region attention guidance"""
    
    def __init__(self, df, img_size=224):
        self.df = df.copy().reset_index(drop=True)
        self.img_size = img_size
        
        sample_paths = self.df['Image Path'].sample(min(200, len(self.df)), random_state=42).values
        missing = [p for p in sample_paths if not os.path.exists(p)]
        if missing:
            raise FileNotFoundError(f"❌ Missing {len(missing)} images!")
        
        print(f"📦 AttentionGuidedDataset: {len(self.df)} samples")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['Image Path']
        
        img = Image.open(img_path).convert('L')
        img = img.resize((self.img_size, self.img_size), Image.LANCZOS)
        img = np.array(img, dtype=np.float32) / 255.0
        
        # Get region masks
        regions = multi_region_segmentation(img)
        region_masks = np.stack([regions[k] for k in sorted(regions.keys())], axis=0)
        
        # Two augmented views
        view1 = apply_augmentation(img.copy())
        view2 = apply_augmentation(img.copy())
        
        view1 = torch.tensor(view1[None, ...], dtype=torch.float32)
        view2 = torch.tensor(view2[None, ...], dtype=torch.float32)
        region_masks = torch.tensor(region_masks, dtype=torch.float32)
        
        return view1, view2, region_masks


class ClassificationDataset(Dataset):
    """Classification dataset"""
    
    def __init__(self, df, disease_categories, img_size=224):
        self.df = df.copy().reset_index(drop=True)
        self.disease_categories = disease_categories
        self.img_size = img_size
        
        print(f"📦 ClassificationDataset: {len(self.df)} samples")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        img = Image.open(row['Image Path']).convert('L')
        img = img.resize((self.img_size, self.img_size), Image.LANCZOS)
        img = np.array(img, dtype=np.float32) / 255.0
        img = torch.tensor(img[None, ...], dtype=torch.float32)
        
        labels = torch.tensor([row[d] for d in self.disease_categories], dtype=torch.float32)
        
        return img, labels

print("✅ Dataset classes defined")

In [ ]:
# ============================================
# 🏗️ Step 8: Attention-Guided Encoder
# ============================================

def conv_block(in_c, out_c, kernel=3, stride=1, padding=1):
    return nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel, stride, padding),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True)
    )

def residual_block(channels):
    return nn.Sequential(
        conv_block(channels, channels),
        conv_block(channels, channels)
    )

class RegionAttention(nn.Module):
    """Region-specific attention module"""
    def __init__(self, in_channels, num_regions=6):
        super().__init__()
        self.num_regions = num_regions
        self.attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(in_channels, num_regions),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x, region_masks):
        """Apply region-aware attention"""
        batch_size = x.size(0)
        
        attention_weights = self.attention(x)
        
        # Resize region masks to match feature map size
        h, w = x.shape[2:]
        region_masks_resized = F.interpolate(
            region_masks, size=(h, w), mode='bilinear', align_corners=False
        )
        
        # Apply region-aware weighting
        weighted_x = x.clone()
        for i in range(self.num_regions):
            region_mask = region_masks_resized[:, i:i+1, :, :]
            weight = attention_weights[:, i:i+1].view(batch_size, 1, 1, 1)
            weighted_x = weighted_x + weight * region_mask * x
        
        return weighted_x / self.num_regions


class AttentionEncoder(nn.Module):
    """Encoder with segmentation-guided attention"""
    def __init__(self, in_channels=1, feat_dim=256, num_regions=6):
        super().__init__()
        self.num_regions = num_regions
        
        self.features = nn.Sequential(
            conv_block(in_channels, 64), residual_block(64), nn.MaxPool2d(2),
            conv_block(64, 128), residual_block(128), nn.MaxPool2d(2),
            conv_block(128, 256), residual_block(256), residual_block(256), nn.MaxPool2d(2),
            conv_block(256, 512), residual_block(512), residual_block(512), nn.MaxPool2d(2),
            conv_block(512, 512), residual_block(512), nn.AdaptiveAvgPool2d((1, 1))
        )
        
        self.region_attention_1 = RegionAttention(128, num_regions)
        self.region_attention_2 = RegionAttention(256, num_regions)
        self.region_attention_3 = RegionAttention(512, num_regions)
        
        self.fc = nn.Sequential(
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, feat_dim)
        )
    
    def forward(self, x, region_masks=None):
        # If no attention guidance, use standard encoder
        if region_masks is None:
            x = self.features(x)
            return self.fc(x.view(x.size(0), -1))
        
        # Apply attention at intermediate layers
        x = self.features[0:3](x)
        x = self.region_attention_1(x, region_masks)
        
        x = self.features[3:6](x)
        x = self.region_attention_2(x, region_masks)
        
        x = self.features[6:11](x)
        x = self.region_attention_3(x, region_masks)
        
        x = self.features[11:](x)
        return self.fc(x.view(x.size(0), -1))

class ProjectionHead(nn.Module):
    def __init__(self, feat_dim=256, proj_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(feat_dim, feat_dim), nn.BatchNorm1d(feat_dim), nn.ReLU(),
            nn.Linear(feat_dim, proj_dim)
        )
    def forward(self, x):
        return self.net(x)

class Decoder(nn.Module):
    def __init__(self, feat_dim=256, img_size=224):
        super().__init__()
        self.init_size = img_size // 32
        self.fc = nn.Sequential(
            nn.Linear(feat_dim, 256 * self.init_size * self.init_size), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 4, 2, 1), nn.BatchNorm2d(16), nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 4, 2, 1), nn.Sigmoid()
        )
    def forward(self, z):
        x = self.fc(z)
        return self.decoder(x.view(z.size(0), 256, self.init_size, self.init_size))

class Classifier(nn.Module):
    def __init__(self, feat_dim=256, num_classes=14):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(feat_dim, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        return self.net(x)

encoder = AttentionEncoder(feat_dim=cfg.feat_dim, num_regions=cfg.num_regions).to(cfg.device)
proj_head = ProjectionHead(cfg.feat_dim, cfg.proj_dim).to(cfg.device)
decoder = Decoder(cfg.feat_dim, cfg.img_size).to(cfg.device)

total_params = sum(p.numel() for m in [encoder, proj_head, decoder] for p in m.parameters())
print(f"✅ Attention-Guided Models initialized ({total_params:,} parameters)")

In [ ]:
# ============================================
# 🔥 Step 9: Loss Functions
# ============================================

def nt_xent_loss(z1, z2, temperature=0.1):
    device = z1.device
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)
    batch_size = z1.shape[0]
    representations = torch.cat([z1, z2], dim=0)
    similarity = torch.matmul(representations, representations.T) / temperature
    mask = torch.eye(2 * batch_size, dtype=torch.bool, device=device)
    similarity = similarity.masked_fill(mask, -float('inf'))
    labels = torch.cat([torch.arange(batch_size) + batch_size,
                        torch.arange(batch_size)]).to(device)
    return F.cross_entropy(similarity, labels)

print("✅ Loss functions defined")

In [ ]:
# ============================================
# 📊 Step 10: Create Data Loaders
# ============================================

df_shuffled = df_labels.sample(frac=1, random_state=42).reset_index(drop=True)
train_size = int(0.8 * len(df_shuffled))
train_df = df_shuffled[:train_size]
val_df = df_shuffled[train_size:]

train_pretrain_ds = AttentionGuidedDataset(train_df, img_size=cfg.img_size)
train_class_ds = ClassificationDataset(train_df, DISEASE_CATEGORIES, cfg.img_size)
val_class_ds = ClassificationDataset(val_df, DISEASE_CATEGORIES, cfg.img_size)

pretrain_loader = DataLoader(
    train_pretrain_ds, batch_size=cfg.batch_size, shuffle=True,
    num_workers=cfg.num_workers, pin_memory=True, drop_last=True
)
train_loader = DataLoader(
    train_class_ds, batch_size=cfg.batch_size, shuffle=True,
    num_workers=cfg.num_workers, pin_memory=True, drop_last=True
)
val_loader = DataLoader(
    val_class_ds, batch_size=cfg.batch_size, shuffle=False,
    num_workers=cfg.num_workers, pin_memory=True
)

print(f"✅ DataLoaders ready")

In [ ]:
# ============================================
# 🚀 Step 11: Attention-Guided SSL Pretraining
# ============================================

optimizer_ssl = torch.optim.Adam(
    list(encoder.parameters()) + list(proj_head.parameters()) + list(decoder.parameters()),
    lr=cfg.lr_pretrain, weight_decay=1e-4
)

ssl_history = {'loss': [], 'contrastive': [], 'reconstruction': []}
START_EPOCH = 1

if RESUME_SSL_PRETRAINING:
    ckpt_file = find_latest_checkpoint(f'{OPTION_NAME}_ssl') if SSL_CHECKPOINT_FILE == "latest" else SSL_CHECKPOINT_FILE
    if ckpt_file:
        checkpoint = load_checkpoint(ckpt_file)
        if checkpoint:
            encoder.load_state_dict(checkpoint['encoder'])
            proj_head.load_state_dict(checkpoint['proj_head'])
            decoder.load_state_dict(checkpoint['decoder'])
            if 'optimizer' in checkpoint: optimizer_ssl.load_state_dict(checkpoint['optimizer'])
            ssl_history = checkpoint.get('ssl_history', ssl_history)
            START_EPOCH = checkpoint['epoch'] + 1
            print(f"🔄 Resuming from epoch {START_EPOCH}")
    else:
        print("⚠️ No checkpoint found. Starting fresh.")

if START_EPOCH > cfg.pretrain_epochs:
    print(f"✅ SSL Pretraining already complete")
else:
    print(f"\n🧠 Starting Option 5: Attention-Guided SSL Pretraining")
    print(f"   Epochs: {START_EPOCH} → {cfg.pretrain_epochs}")
    print("=" * 60)
    
    for epoch in range(START_EPOCH, cfg.pretrain_epochs + 1):
        encoder.train()
        proj_head.train()
        decoder.train()
        
        total_loss = total_cont = total_recon = 0
        
        pbar = tqdm(pretrain_loader, desc=f"Epoch {epoch}/{cfg.pretrain_epochs}")
        for view1, view2, region_masks in pbar:
            view1 = view1.to(cfg.device)
            view2 = view2.to(cfg.device)
            region_masks = region_masks.to(cfg.device)
            
            optimizer_ssl.zero_grad()
            
            z1 = encoder(view1, region_masks)
            z2 = encoder(view2, region_masks)
            
            p1, p2 = proj_head(z1), proj_head(z2)
            cont_loss = nt_xent_loss(p1, p2, cfg.temperature)
            
            rec1, rec2 = decoder(z1), decoder(z2)
            recon_loss = (F.mse_loss(rec1, view1) + F.mse_loss(rec2, view2)) / 2
            
            loss = cont_loss + 0.5 * recon_loss
            loss.backward()
            optimizer_ssl.step()
            
            total_loss += loss.item()
            total_cont += cont_loss.item()
            total_recon += recon_loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        n = len(pretrain_loader)
        ssl_history['loss'].append(total_loss / n)
        ssl_history['contrastive'].append(total_cont / n)
        ssl_history['reconstruction'].append(total_recon / n)
        
        print(f"Epoch {epoch}: Loss={total_loss/n:.4f}, Cont={total_cont/n:.4f}, Recon={total_recon/n:.4f}")
        
        save_checkpoint({
            'epoch': epoch, 'encoder': encoder.state_dict(),
            'proj_head': proj_head.state_dict(), 'decoder': decoder.state_dict(),
            'optimizer': optimizer_ssl.state_dict(), 'ssl_history': ssl_history,
        }, f'{OPTION_NAME}_ssl_latest.pth')
        save_checkpoint({
            'epoch': epoch, 'encoder': encoder.state_dict(),
            'proj_head': proj_head.state_dict(), 'decoder': decoder.state_dict(),
            'ssl_history': ssl_history,
        }, f'{OPTION_NAME}_ssl_epoch{epoch}.pth')
    
    print("\n✅ Attention-Guided SSL Pretraining Complete!")

In [ ]:
# ============================================
# 📈 Step 12: Plot SSL Curves & Save Model
# ============================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(ssl_history['loss'], 'b-', linewidth=2)
axes[0].set_title('Total Loss')
axes[0].set_xlabel('Epoch')
axes[0].grid(True, alpha=0.3)

axes[1].plot(ssl_history['contrastive'], 'r-', linewidth=2)
axes[1].set_title('Contrastive Loss')
axes[1].set_xlabel('Epoch')
axes[1].grid(True, alpha=0.3)

axes[2].plot(ssl_history['reconstruction'], 'g-', linewidth=2)
axes[2].set_title('Reconstruction Loss')
axes[2].set_xlabel('Epoch')
axes[2].grid(True, alpha=0.3)

plt.suptitle('Option 5: Attention-Guided SSL Training', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('option5_ssl_curves.png', dpi=150, bbox_inches='tight')
plt.show()

torch.save({
    'encoder': encoder.state_dict(),
    'proj_head': proj_head.state_dict(),
    'decoder': decoder.state_dict(),
}, 'option5_ssl_pretrained.pth')

print("💾 Pretrained model saved: option5_ssl_pretrained.pth")

In [ ]:
# ============================================
# 🎯 Step 13-15: Fine-tuning, Evaluation, & Summary
# ============================================

for param in encoder.parameters():
    param.requires_grad = False
encoder.eval()

classifier = Classifier(cfg.feat_dim, len(DISEASE_CATEGORIES)).to(cfg.device)

pos_counts = train_df[DISEASE_CATEGORIES].sum().values
neg_counts = len(train_df) - pos_counts
pos_weights = torch.tensor(neg_counts / (pos_counts + 1e-6), dtype=torch.float32).to(cfg.device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
optimizer_ft = torch.optim.Adam(classifier.parameters(), lr=cfg.lr_finetune, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, 'max', patience=5, factor=0.5)

finetune_history = {'train_loss': [], 'train_auc': [], 'val_loss': [], 'val_auc': []}
best_val_auc = 0
FINETUNE_START_EPOCH = 1

if RESUME_FINETUNING:
    ckpt_file = find_latest_checkpoint(f'{OPTION_NAME}_finetune') if FINETUNE_CHECKPOINT_FILE == "latest" else FINETUNE_CHECKPOINT_FILE
    if ckpt_file:
        checkpoint = load_checkpoint(ckpt_file)
        if checkpoint:
            classifier.load_state_dict(checkpoint['classifier'])
            if 'optimizer' in checkpoint: optimizer_ft.load_state_dict(checkpoint['optimizer'])
            if 'scheduler' in checkpoint: scheduler.load_state_dict(checkpoint['scheduler'])
            finetune_history = checkpoint.get('finetune_history', finetune_history)
            best_val_auc = checkpoint.get('best_val_auc', 0)
            FINETUNE_START_EPOCH = checkpoint['epoch'] + 1
            print(f"🔄 Resuming fine-tuning from epoch {FINETUNE_START_EPOCH}")
    else:
        print("⚠️ No checkpoint found. Starting fresh.")

if FINETUNE_START_EPOCH > cfg.finetune_epochs:
    print(f"✅ Fine-tuning already complete")
else:
    print("🎯 Starting Fine-tuning")
    print(f"   Epochs: {FINETUNE_START_EPOCH} → {cfg.finetune_epochs}")
    print("=" * 50)
    
    for epoch in range(FINETUNE_START_EPOCH, cfg.finetune_epochs + 1):
        classifier.train()
        train_loss = 0
        train_preds, train_targets = [], []
        
        for images, targets in tqdm(train_loader, desc=f"Train {epoch}/{cfg.finetune_epochs}"):
            images, targets = images.to(cfg.device), targets.to(cfg.device)
            optimizer_ft.zero_grad()
            with torch.no_grad():
                features = encoder(images)
            logits = classifier(features)
            loss = criterion(logits, targets)
            loss.backward()
            optimizer_ft.step()
            train_loss += loss.item()
            train_preds.append(torch.sigmoid(logits).detach().cpu())
            train_targets.append(targets.cpu())
        
        classifier.eval()
        val_loss = 0
        val_preds, val_targets = [], []
        
        with torch.no_grad():
            for images, targets in val_loader:
                images, targets = images.to(cfg.device), targets.to(cfg.device)
                features = encoder(images)
                logits = classifier(features)
                val_loss += criterion(logits, targets).item()
                val_preds.append(torch.sigmoid(logits).cpu())
                val_targets.append(targets.cpu())
        
        train_preds = torch.cat(train_preds).numpy()
        train_targets = torch.cat(train_targets).numpy()
        val_preds = torch.cat(val_preds).numpy()
        val_targets = torch.cat(val_targets).numpy()
        
        train_auc = np.mean([roc_auc_score(train_targets[:, i], train_preds[:, i]) 
                             for i in range(len(DISEASE_CATEGORIES)) 
                             if len(np.unique(train_targets[:, i])) > 1])
        val_auc = np.mean([roc_auc_score(val_targets[:, i], val_preds[:, i]) 
                           for i in range(len(DISEASE_CATEGORIES)) 
                           if len(np.unique(val_targets[:, i])) > 1])
        
        finetune_history['train_loss'].append(train_loss / len(train_loader))
        finetune_history['train_auc'].append(train_auc)
        finetune_history['val_loss'].append(val_loss / len(val_loader))
        finetune_history['val_auc'].append(val_auc)
        scheduler.step(val_auc)
        
        print(f"Epoch {epoch}: Train AUC={train_auc:.4f}, Val AUC={val_auc:.4f}")
        
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            torch.save({'encoder': encoder.state_dict(), 'classifier': classifier.state_dict()}, 
                       f'{OPTION_NAME}_best_model.pth')
            print(f"  ✅ Best model saved! Val AUC: {val_auc:.4f}")
        
        save_checkpoint({
            'epoch': epoch, 'classifier': classifier.state_dict(),
            'optimizer': optimizer_ft.state_dict(), 'scheduler': scheduler.state_dict(),
            'finetune_history': finetune_history, 'best_val_auc': best_val_auc,
        }, f'{OPTION_NAME}_finetune_latest.pth')
        save_checkpoint({
            'epoch': epoch, 'classifier': classifier.state_dict(),
            'finetune_history': finetune_history, 'best_val_auc': best_val_auc,
        }, f'{OPTION_NAME}_finetune_epoch{epoch}.pth')
    
    print(f"\n🏆 Best Validation AUC: {best_val_auc:.4f}")

In [ ]:
# ============================================
# 📊 Step 16: Final Evaluation & Results
# ============================================

checkpoint = torch.load(f'{OPTION_NAME}_best_model.pth')
encoder.load_state_dict(checkpoint['encoder'])
classifier.load_state_dict(checkpoint['classifier'])
encoder.eval()
classifier.eval()

all_preds, all_targets = [], []
with torch.no_grad():
    for images, targets in tqdm(val_loader, desc="Evaluating"):
        images = images.to(cfg.device)
        features = encoder(images)
        logits = classifier(features)
        all_preds.append(torch.sigmoid(logits).cpu())
        all_targets.append(targets)

all_preds = torch.cat(all_preds).numpy()
all_targets = torch.cat(all_targets).numpy()

print("\n📊 Per-Disease AUC Scores:")
print("=" * 40)
auc_scores = []
for i, disease in enumerate(DISEASE_CATEGORIES):
    if len(np.unique(all_targets[:, i])) > 1:
        auc = roc_auc_score(all_targets[:, i], all_preds[:, i])
        auc_scores.append((disease, auc))
        print(f"{disease:20s}: {auc:.4f}")

mean_auc = np.mean([a for _, a in auc_scores])
print(f"\n{'Mean AUC':20s}: {mean_auc:.4f}")

auc_scores.sort(key=lambda x: x[1], reverse=True)
diseases, aucs = zip(*auc_scores)

plt.figure(figsize=(12, 6))
colors = ['green' if a >= 0.7 else 'orange' if a >= 0.6 else 'red' for a in aucs]
plt.barh(diseases, aucs, color=colors, alpha=0.8)
plt.axvline(0.5, color='red', linestyle='--', alpha=0.5, label='Random')
plt.axvline(mean_auc, color='blue', linestyle='--', alpha=0.7, label=f'Mean: {mean_auc:.3f}')
plt.xlabel('AUC Score')
plt.title('Option 5: Per-Disease AUC Performance (Attention-Guided)', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig(f'{OPTION_NAME}_auc_performance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "=" * 70)
print("🧠 OPTION 5: SEGMENTATION-GUIDED ATTENTION SUMMARY")
print("=" * 70)
print(f"Method: Region-Specific Attention in Encoder Architecture")
print(f"\nKey Advantages:")
print(f"  ✅ Learnable region-specific attention weights")
print(f"  ✅ Network learns which regions matter for SSL")
print(f"  ✅ Combines supervised segmentation with unsupervised SSL")
print(f"  ✅ More flexible than hard cropping")
print(f"  ✅ Emergent focus on pathological patterns")
print(f"\n🏆 Final Mean AUC: {mean_auc:.4f}")
print("=" * 70)